## Performance Tuning: SQL & Table Optimization

**The Situation:** Leadership wants dashboards, predictive models, and AI agents ready by Friday. Your plane IoT data is growing fast, and queries that worked yesterday are timing out today.

**The Problem:** Slow queries = higher cost, missed deadlines, and angry leadership

**The Solution:** Get familiar with both SQL and table optimization techniques to get sub-second query times.

---

## What You'll Learn

✅ **SQL Optimization:** Predicate pushdown, join strategies, broadcast hints  
✅ **Liquid Clustering:** Automatic data layout optimization  
✅ **Materialized Views:** Pre-compute expensive aggregations  
✅ **Query Profile:** Analyze query execution on SQL Warehouses  

---

## Prerequisites

- Completed Day 1 & 2
- `sensor_bronze`, `dim_factories`, `dim_devices` tables loaded
- SQL Warehouse or cluster running

---

**References:**
- [Delta Lake Performance](https://docs.databricks.com/en/delta/tune-file-layout.html)
- [Liquid Clustering](https://docs.databricks.com/en/delta/clustering.html)
- [Query Optimization](https://docs.databricks.com/en/optimizations/)


In [0]:
# Configuration
import re

catalog = "dwx_airops_insights_platform_dev_working"
source_schema = "db_crash_course"  # Shared schema to read from
username = spark.sql("SELECT current_user()").collect()[0][0]
username_base = username.split('@')[0]  # Extract username before @ symbol
target_schema = re.sub(r'[^a-zA-Z0-9_]', '_', username_base)  # Replace special chars with _

# Create target schema if it doesn't exist
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{target_schema}")

print(f"✅ Using catalog: {catalog}")
print(f"📖 Reading from schema: {source_schema} (shared)")
print(f"✍️  Writing to schema: {target_schema} (your personal schema)")

## Part 1: Understanding Performance Bottlenecks

### Common Performance Killers

| Problem | Impact | Solution |
|---------|--------|----------|
| 🐌 **Small Files** | Too many file opens | OPTIMIZE |
| 🐌 **Full Table Scans** | Read entire table | Liquid Clustering, predicates |
| 🐌 **Data Shuffle** | Network overhead | Broadcast joins |
| 🐌 **Wrong Join Type** | Memory spills | Join hints |
| 🐌 **Repeated Computation** | Wasted resources | Materialized views |
| 🐌 **Inefficient Predicates** | No pushdown | Proper filters |

### Performance Toolkit

**SQL Optimization:**
- Predicate pushdown (filter early)
- Join hints (BROADCAST, SHUFFLE_HASH)
- Proper WHERE clause design
- Query Profile analysis

**Table Optimization:**
- File compaction (OPTIMIZE)
- Liquid Clustering (automatic data layout optimization)
- Deletion Vectors (fast updates)

**Query Results:**
- Materialized Views
- Caching


## Part 2: Creating a "Bad" Table for Demonstration

Let's intentionally create a poorly optimized table with:
- Many small files (simulating streaming ingestion)
- Random data layout (no locality)
- No optimization

This represents what happens in real production systems without proper maintenance!


In [0]:
# Step 1: Create unoptimized table with random layout in your target schema
spark.sql(f"DROP TABLE IF EXISTS {catalog}.{target_schema}.sensor_unoptimized")

spark.sql(f"""
CREATE TABLE {catalog}.{target_schema}.sensor_unoptimized
AS
SELECT 
    device_id,
    trip_id,
    factory_id,
    model_id,
    timestamp,
    airflow_rate,
    rotation_speed,
    air_pressure,
    temperature,
    delay,
    density
FROM {catalog}.{source_schema}.sensor_bronze
ORDER BY RAND()  -- Random order = worst case for data locality!
LIMIT 200000  -- Use subset for demo
""")

print("✅ Created unoptimized table with random layout")


In [0]:
# Step 2: Simulate many small files (like streaming writes)
# This is what happens with continuous ingestion without auto-compaction

for i in range(15):  # Create 15 small file batches
    spark.sql(f"""
    INSERT INTO {catalog}.{target_schema}.sensor_unoptimized
    SELECT 
        device_id,
        trip_id,
        factory_id,
        model_id,
        timestamp,
        airflow_rate,
        rotation_speed,
        air_pressure,
        temperature,
        delay,
        density
    FROM {catalog}.{source_schema}.sensor_bronze
    WHERE MOD(device_id, 15) = {i}
    LIMIT 800
    """)

print("✅ Created many small files (simulating poor ingestion patterns)")


In [0]:
# Check table statistics - look at the file count!
display(spark.sql(f"""
DESCRIBE DETAIL {catalog}.{target_schema}.sensor_unoptimized
""").select("numFiles", "sizeInBytes", "minReaderVersion", "minWriterVersion"))


### 🔍 What to Look For:

- **numFiles**: High number (hundreds of thousands or millions) = Performance problem!
- **sizeInBytes**: Total size, but spread across too many files

**Problem:** Every query must:
1. List all files
2. Open each file
3. Read metadata
4. Scan for relevant data

With many small files, overhead dominates actual work!


## Part 3: SQL Optimization - Predicate Pushdown

**Key Concept:** Push filters as close to the data as possible.

### Comparing Query Performance

Let's compare two approaches to filtering - one that prevents optimization and one that enables it.

#### ❌ Without Predicate Pushdown (Bad)

Using a function on the column prevents statistics-based filtering:

In [0]:
import time

# BAD: Using SUBSTRING on timestamp prevents predicate pushdown
start = time.time()

result_bad = spark.sql(f"""
SELECT 
    device_id,
    factory_id,
    AVG(temperature) as avg_temp
FROM {catalog}.{target_schema}.sensor_unoptimized
WHERE SUBSTRING(CAST(timestamp AS STRING), 1, 10) >= DATE_SUB(CURRENT_DATE(), 7)
GROUP BY device_id, factory_id
""")

count_bad = result_bad.count()
time_bad = time.time() - start

print(f"❌ BAD Query Time: {time_bad:.2f} seconds")
print(f"   Results: {count_bad} rows")
print(f"   Problem: Function on column prevents statistics-based filtering!")


#### ✅ With Predicate Pushdown (Good)

Direct filter on timestamp column enables predicate pushdown:

In [0]:
# GOOD: Direct filter on timestamp enables predicate pushdown
start = time.time()

result_good = spark.sql(f"""
SELECT 
    device_id,
    factory_id,
    AVG(temperature) as avg_temp
FROM {catalog}.{target_schema}.sensor_unoptimized
WHERE timestamp >= DATE_SUB(CURRENT_DATE(), 7)
GROUP BY device_id, factory_id
""")

count_good = result_good.count()
time_good = time.time() - start

print(f"✅ GOOD Query Time: {time_good:.2f} seconds")
print(f"   Results: {count_good} rows")
print(f"   Benefit: Delta uses file statistics to skip irrelevant files!")

# Show improvement
if time_good > 0 and time_bad > 0:
    speedup = time_bad / time_good
    print(f"\n🚀 Speedup: {speedup:.1f}x faster with predicate pushdown!")

### Best Practices for Predicate Pushdown

**DO:**
```sql
WHERE timestamp >= '2024-01-01'  -- Direct column comparison
WHERE device_id IN (1, 2, 3)     -- Direct value check
WHERE factory_id = 'A06'          -- Equality on column
```

**DON'T:**
```sql
WHERE DATE(timestamp) = '2024-01-01'      -- Function prevents pushdown
WHERE SUBSTRING(device_id, 1, 2) = '10'   -- Function on column
WHERE UPPER(factory_id) = 'A06'           -- Transformation blocks optimization
```

## Part 4: SQL Optimization - Join Strategies

### Understanding Join Types

| Join Type | Best For | Cost |
|-----------|----------|------|
| **Broadcast Join** | Small table (< 10MB) | Low - no shuffle |
| **Shuffle Hash Join** | Large tables | High - shuffle both |
| **Sort Merge Join** | Large sorted tables | Medium |

### Comparing Join Strategies

Let's compare queries with and without join hints to see the performance impact.

#### ⚠️ Without Hint - Spark Auto-Selects Join Strategy

Without a hint we let Spark choose the join strategy automatically

#### ✅ With Broadcast Hint - Force Efficient Join

Use the BROADCAST hint to force an efficient join strategy:

In [0]:
# WITH BROADCAST hint - Force efficient join strategy
start = time.time()

result_broadcast = spark.sql(f"""
SELECT /*+ BROADCAST(f) */
    s.device_id,
    f.factory_name,
    f.region,
    AVG(s.temperature) as avg_temp,
    COUNT(*) as reading_count
FROM {catalog}.{target_schema}.sensor_unoptimized s
JOIN {catalog}.{source_schema}.dim_factories f
  ON s.factory_id = f.factory_id
WHERE s.timestamp >= CURRENT_DATE() - INTERVAL 7 DAYS
GROUP BY s.device_id, f.factory_name, f.region
""")

count_broadcast = result_broadcast.count()
time_broadcast = time.time() - start

print(f"✅ Query Time (Broadcast): {time_broadcast:.2f} seconds")
print(f"   Small dimension table broadcast to all nodes - no shuffle!")

### Best Practices for Join Optimization

**BROADCAST when:**
- Dimension table < 10MB
- Reference data (factories, models, devices)
- Lookup tables

**Let Spark choose when:**
- Both tables are large
- Join cardinality is unknown
- Adaptive Query Execution is enabled (default)

### 💡 Join Optimization Rules

**BROADCAST when:**
- Dimension table < 10MB
- Reference data (factories, models, devices)
- Lookup tables

**Let Spark choose when:**
- Both tables are large
- Join cardinality is unknown
- Adaptive Query Execution is enabled (default)


In [0]:
# Check current file situation
detail_before = spark.sql(f"""
DESCRIBE DETAIL {catalog}.{target_schema}.sensor_unoptimized
""").select("numFiles", "sizeInBytes").collect()[0]

files_before = detail_before['numFiles']
size_mb = detail_before['sizeInBytes'] / 1024 / 1024

print(f"📊 Before Optimization:")
print(f"   Files: {files_before}")
print(f"   Size: {size_mb:.2f} MB")
print(f"   Avg file size: {size_mb/files_before:.2f} MB")
print(f"\n   Status: {'🔴 Too many small files!' if files_before > 10 else '🟢 OK'}")


In [0]:
# Run OPTIMIZE to compact files
start = time.time()

spark.sql(f"""
OPTIMIZE {catalog}.{target_schema}.sensor_unoptimized
""")

# Check the new file count
detail_after = spark.sql(f"""
DESCRIBE DETAIL {catalog}.{target_schema}.sensor_unoptimized
""").select("numFiles", "sizeInBytes").collect()[0]

files_after = detail_after['numFiles']
size_mb = detail_after['sizeInBytes'] / 1024 / 1024

print(f"📊 After Optimization:")
print(f"   Files: {files_after}")
print(f"   Size: {size_mb:.2f} MB")
print(f"   Avg file size: {size_mb/files_after:.2f} MB")


## Part 5: Table Optimization - Liquid Clustering

**Liquid Clustering** is Delta Lake's automatic data layout optimization. Unity Catalog can automatically cluster all of your tables, [described here](https://docs.databricks.com/aws/en/optimizations/predictive-optimization), or you can manually cluster them as shown below

### Why Liquid Clustering?
- ✅ Automatic optimization during writes
- ✅ Adapts to changing access patterns
- ✅ No manual maintenance required
- ✅ Combines compaction + data layout

### How Data Skipping Works

**Without Clustering:**
```
File 1: devices 1,5,10,15,20     <- Must read
File 2: devices 2,3,8,12,19      <- Must read  
File 3: devices 4,7,9,11,14      <- Must read
```
Query for device_id = 5 must read ALL files!

**With Liquid Clustering on device_id:**
```
File 1: devices 1,2,3,4,5        <- Read this (automatically organized!)
File 2: devices 7,8,9,10,11      <- SKIP
File 3: devices 12,14,15,19,20   <- SKIP
```
Query for device_id = 5 only reads File 1!

### Choosing Clustering Columns

✅ **Good candidates:**
- High cardinality (device_id, timestamp)
- Frequently in WHERE clauses
- Used in joins
- Common GROUP BY columns

❌ **Bad candidates:**
- Low cardinality (status: active/inactive)
- Rarely filtered

**Rule:** 2-4 columns maximum, order matters (most selective first)

In [0]:
# First, let's create a clustered version of the unoptimized table
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog}.{target_schema}.sensor_clustered
CLUSTER BY (device_id, timestamp)
AS SELECT * FROM {catalog}.{source_schema}.sensor_bronze
""")

print("✅ Created table with Liquid Clustering on (device_id, timestamp)")

In [0]:
import time

# BAD: Using SUBSTRING on timestamp prevents predicate pushdown
start = time.time()

result_bad = spark.sql(f"""
SELECT 
    device_id,
    factory_id,
    AVG(temperature) as avg_temp
FROM {catalog}.{target_schema}.sensor_clustered
GROUP BY device_id, factory_id
""")

count_bad = result_bad.count()
time_bad = time.time() - start

print(f"❌ BAD Query Time: {time_bad:.2f} seconds")
print(f"   Results: {count_bad} rows")
print(f"   Problem: No data skipping!")


In [0]:
# GOOD: Direct filter on timestamp enables predicate pushdown
start = time.time()

result_good = spark.sql(f"""
SELECT 
    device_id,
    factory_id,
    AVG(temperature) as avg_temp
FROM {catalog}.{target_schema}.sensor_clustered
GROUP BY device_id, factory_id
""")

count_good = result_good.count()
time_good = time.time() - start

print(f"✅ GOOD Query Time: {time_good:.2f} seconds")
print(f"   Results: {count_good} rows")
print(f"   Benefit: Delta uses file statistics to skip irrelevant files!")

# Show improvement
if time_good > 0 and time_bad > 0:
    speedup = time_bad / time_good
    print(f"\n🚀 Speedup: {speedup:.1f}x faster with predicate pushdown!")

## Part 6. Caching Strategies

**Caching** keeps frequently accessed data in memory for instant access.

### Types of Caching:

1. **DataFrame Cache**: Temporary, session-specific
2. **Delta Cache**: Disk-based, persists across queries
3. **Result Cache**: Caches query results

### When to Use Caching:

✅ Dimension tables (small, frequently joined)  
✅ Reference data  
✅ Iterative ML training  
✅ Dashboard data sources  

❌ Don't cache:
- Large fact tables (waste of memory)
- Rarely accessed data
- Data that changes frequently


In [0]:
# Cache frequently used dimension tables
# These are joined in almost every query!

spark.sql(f"CACHE TABLE {catalog}.{source_schema}.dim_factories")
spark.sql(f"CACHE TABLE {catalog}.{source_schema}.dim_models")
spark.sql(f"CACHE TABLE {catalog}.{source_schema}.dim_devices")

print("✅ Cached dimension tables")
print("   Joins with these tables are now instant!")


In [0]:
# Test query with cached dimensions
start = time.time()

result_cached = spark.sql(f"""
SELECT 
    f.factory_name,
    f.region,
    m.model_name,
    m.model_family,
    d.device_id,
    COUNT(DISTINCT s.trip_id) as trip_count,
    AVG(s.temperature) as avg_temp
FROM {catalog}.{source_schema}.sensor_bronze s
JOIN {catalog}.{source_schema}.dim_devices d ON s.device_id = d.device_id
JOIN {catalog}.{source_schema}.dim_factories f ON d.factory_id = f.factory_id
JOIN {catalog}.{source_schema}.dim_models m ON d.model_id = m.model_id
GROUP BY f.factory_name, f.region, m.model_name, m.model_family, d.device_id
""")

display(result_cached)

cached_time = time.time() - start
print(f"\n⏱️  Query time (with cached dimensions): {cached_time:.2f} seconds")
print("✨ Dimension joins are instant - no disk I/O needed!")

## Try This: Using Query Profile on SQL Warehouses

**Query Profile** is your best friend for diagnosing slow queries on SQL Warehouses.

### What is Query Profile?

Query Profile shows you **exactly** what your query is doing:
- Which operations took the longest
- How much data was read
- Where shuffles happened
- Memory spills

### How to Access Query Profile

1. Run a query on a **SQL Warehouse** (not a cluster)
2. After the query completes, click the **"Query Profile"** tab
3. Explore the visual execution plan

### What to Look For

| Problem in Profile | Meaning | Solution |
|-------------------|---------|----------|
| 🔴 **Large Scan** | Reading too much data | Add Liquid Clustering, better filters |
| 🔴 **Shuffle** | Data moving between nodes | Use broadcast joins for small tables |
| 🔴 **Spill to Disk** | Out of memory | Increase warehouse size or optimize query |
| 🔴 **Many Tasks** | Too many small files | Run OPTIMIZE |

### Example Workflow

```
1. Query is slow (10+ seconds) ❌
2. Check Query Profile → See "Large Scan"
3. Add Liquid Clustering to table
4. Re-run query → 2 seconds ✅
```

**Learn more:** [Query Profile Documentation](https://docs.databricks.com/aws/en/sql/user/queries/query-profile)

**💡 Pro Tip:** Query Profile only works on SQL Warehouses, not all-purpose or job clusters. If you're running notebooks on a cluster, switch to a SQL Warehouse to use this feature.